# Проект для "Викишоп" с BERT

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.

Алгоритм решения выглядит так:
1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

## Описание данных
Данные находятся в файле /datasets/toxic_comments.csv.
Столбец text в нём содержит текст комментария, а toxic — целевой признак.

In [7]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import re
import torch
import transformers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
class_balance = data['toxic'].value_counts(normalize=True)
print(class_balance)

0    0.898388
1    0.101612
Name: toxic, dtype: float64


*Выходные данные метода value_counts показывают, что примерно ***89.84%*** комментариев в наборе данных помечены как ***нетоксичные (класс 0)***, а около ***10.16%*** — как ***токсичные (класс 1)***. Вот некоторые ***выводы и последствия*** для этого распределения классов:*

**Классовый дисбаланс**

- Набор данных страдает от дисбаланса классов, который часто встречается в реальных наборах данных, особенно в таких контекстах, как обнаружение токсичности, где токсичные комментарии встречаются реже, чем нетоксичные.

**Влияние на обучение модели**

- ***Склонность к классу большинства***: модели машинного обучения, обученные на этом наборе данных, могут быть смещены в сторону класса большинства, чаще прогнозируя нетоксичность, поскольку это доминирующий класс.

**Заключение**

- Классовый дисбаланс необходимо устранять в ходе обучения и оценки модели, чтобы построить модель, которая точно идентифицирует токсичные комментарии, не будучи подавленным классом большинства.
- Следует внимательно отнестись к выбору моделей машинного обучения и показателей оценки, чтобы справедливо оценить производительность модели и гарантировать ее полезность в реальных условиях.

In [5]:
data = data.drop(columns=['Unnamed: 0'])

# Проверка на наличие пропущенных значений
print(data.isnull().sum())

text     0
toxic    0
dtype: int64


In [6]:
def clean_text(text):
    # Удалить символы новой строки
    text = re.sub(r"\n", " ", text)
    
    # Удалить URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # Удалить ссылки на пользователя @ и «#» из твита.
    text = re.sub(r'\@\w+|\#','', text)
    
    # Удалить специальные объекты HTML (например, &amp;)
    text = re.sub(r'\&\w*;', '', text)
    
    # Удалить тикеры
    text = re.sub(r'\$\w*', '', text)
    
    # Удалить все цифры, но также можно сохранить их в зависимости от варианта использования.
    text = re.sub(r'\d+', '', text)
    
    # Удалить лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Преобразовать текст в нижний регистр
    text = text.lower()
    
    return text

data['cleaned_text'] = data['text'].apply(clean_text)
data.head(10)

,text,toxic,cleaned_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww! he matches this background colour i'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man, i'm really not trying to edit war. it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,""" more i can't make any real suggestions on im..."
4,"You, sir, are my hero. Any chance you remember...",0,"you, sir, are my hero. any chance you remember..."
5,"""\n\nCongratulations from me as well, use the ...",0,""" congratulations from me as well, use the too..."
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word 'nonsense' was offensive to ...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


In [7]:
# Инициализировать токенизатор DistilBERT
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tqdm.pandas()

# Токенизация текста - выполнение занимает 6 минут!!!
tokenized = data['cleaned_text'].progress_apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

# Заполнение (padding) токенизированного текста
max_len = max(len(x) for x in tokenized)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])

# Создание масок внимания (attention masks)
attention_mask = np.where(padded != 0, 1, 0)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
# Проверим, доступен ли графический процессор, и соответствующим образом настроим устройство.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузим предварительно обученную модель DistilBERT и переместим ее в графический процессор.
model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)

# Генерация эмбеддингов с использованием графического процессора
batch_size = 200
embeddings = []
for i in trange(padded.shape[0] // batch_size):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    # Переместим мбеддинги обратно в ЦП для дальнейшей обработки.
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

# Обработаем оставшиеся образцы, если они существуют.
remainder = padded.shape[0] % batch_size
if remainder:
    batch = torch.LongTensor(padded[-remainder:]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[-remainder:]).to(device)

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = np.concatenate(embeddings)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

***45 minutes to run!!!***

In [ ]:
# Сначала, разделим данные на 90% обучения+валидация и 10% тест.
X_train_val, X_test, y_train_val, y_test = train_test_split(features, data['toxic'], test_size=0.1, random_state=42, 
                                                            stratify=data['toxic'])

# Теперь разделим набор 90% обучение+валидация на наборы 80% обучения и 10% валидация.
# Поскольку мы хотим, чтобы 10% исходных данных находились в наборе валидация,
# мы вычисляем процент оставшихся данных, которые необходимо выделить для валидации.
validation_size = 0.1 / 0.9
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=validation_size, random_state=42,
                                                 stratify=y_train_val)

In [ ]:
# Инициализируем переменные для отслеживания наилучшей производительности.
best_f1_score = 0
best_c = 0

# Список значений C, которые стоит попробовать
C_values = [0.01, 0.1, 1, 10, 100]

for C in C_values:
    # Обучим модель логистической регрессии с текущим значением C.
    logistic_regression = LogisticRegression(max_iter=1200, C=C)
    logistic_regression.fit(X_train, y_train)

    # Прогнозирование на наборе валидации
    y_pred_lr = logistic_regression.predict(X_val)

    # Оценим модель, используя оценку F1 в наборе валидации.
    f1_lr = f1_score(y_val, y_pred_lr)
    print(f"F1 Score on the validation set with Logistic Regression (C={C}):", f1_lr)

    # Обновим лучшую производительность, если текущая модель лучше.
    if f1_lr > best_f1_score:
        best_f1_score = f1_lr
        best_c = C

print(f"Лучший результат F1 Score: {best_f1_score} полученный при C={best_c}")

F1 Score on the validation set with Logistic Regression (C=0.01): 0.7066473988439306  
F1 Score on the validation set with Logistic Regression (C=0.1): 0.7313485113835377  
F1 Score on the validation set with Logistic Regression (C=1): 0.7393397524071527  
F1 Score on the validation set with Logistic Regression (C=10): 0.7376373626373627  
F1 Score on the validation set with Logistic Regression (C=100): 0.7375643224699828  

Лучший результат F1 Score: 0.7393397524071527 полученный при C=1

In [ ]:
# Обучение модели CatBoost
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=7, verbose=100,
                                    eval_metric='F1',
                                    random_state=42)
catboost_model.fit(X_train, y_train)

# Прогнозирование на наборе валидации
y_pred_cb = catboost_model.predict(X_val)

# Оценим модель, используя оценку F1 в наборе валидации.
f1_cb = f1_score(y_val, y_pred_cb)
print("Оценка F1 на наборе валидации с помощью CatBoost:", f1_cb)

0:	learn: 0.4584714	total: 901ms	remaining: 7m 29s  
100:	learn: 0.7292981	total: 1m 16s	remaining: 5m 3s  
200:	learn: 0.7838299	total: 2m 28s	remaining: 3m 40s  
300:	learn: 0.8241427	total: 3m 38s	remaining: 2m 24s  
400:	learn: 0.8561454	total: 4m 48s	remaining: 1m 11s  
499:	learn: 0.8820189	total: 5m 56s	remaining: 0us  

Оценка F1 на наборе валидации с помощью CatBoost: 0.7158852344296711

In [ ]:
# Прогнозирование на тестовом наборе
y_pred = logistic_regression.predict(X_test)

# Оценка модели, используя метрику F1.
f1 = f1_score(y_test, y_pred)
print("Оценка F1 на тестовом наборе:", f1)

***Оценка F1 на тестовом наборе: 0.7537993920972645!*** 

Целевая метрика достигнута!